In [1]:
import sys
sys.path.append('../')
from rocky.core import rocky
from rocky.triangle import Triangle
from rocky.model_selection.TriangleTimeSeriesSplit import TriangleTimeSeriesSplit
import pandas as pd
import numpy as np

In [2]:
t = Triangle.from_taylor_ashe()
t.df

development_period,1,2,3,4,5,6,7,8,9,10
accident_period,,,,,,,,,,
2001-01-01,357848.0,1124788.0,1735330.0,2218270.0,2745596.0,3319994.0,3466336.0,3606286.0,3833515.0,3901463.0
2002-01-01,352118.0,1236139.0,2170033.0,3353322.0,3799067.0,4120063.0,4647867.0,4914039.0,5339085.0,NaN
2003-01-01,290507.0,1292306.0,2218525.0,3235179.0,3985995.0,4132918.0,4628910.0,4909315.0,NaN,NaN
2004-01-01,310608.0,1418858.0,2195047.0,3757447.0,4029929.0,4381982.0,4588268.0,NaN,NaN,NaN
2005-01-01,443160.0,1136350.0,2128333.0,2897821.0,3402672.0,3873311.0,NaN,NaN,NaN,NaN
2006-01-01,396132.0,1333217.0,2180715.0,2985752.0,3691712.0,NaN,NaN,NaN,NaN,NaN
2007-01-01,440832.0,1288463.0,2419861.0,3483130.0,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-01,359480.0,1421128.0,2864498.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-01,376686.0,1363294.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Create a Rocky object
rky = rocky()

# add triangle
rocky_inputs_file = r"O:\PARM\Individual folders\A Weaver\Reserving\rocky\rocky-inputs.xlsx"
rky.FromExcel(filename=rocky_inputs_file, sheet_name='triangle', id='paid_loss')
# rky.SampleTri('taylor_ashe')

# add model
rky.AddModel('icrfs', model_class='loglinear', tri=rky.paid_loss, use_cal=True, plot_width=None)

This is for testing and compatability only. Please do not assign much credibility to these estimates for the purposes selecting carried reserves.


In [4]:
# set hyperparameters
rky.icrfs.SetHyperparameters(alpha=0.05, l1_ratio=0.01)
# rky.icrfs.TuneFitHyperparameters(l1_ratio=[0, 0.025, 0.05])

# fit model
rky.icrfs.Fit()

# basic hetero adjustment (one-step, no cross-validation,
# just one parameter per development period)
rky.icrfs.FitHetero()

Fitting hetero adjustment: (Step/RMSE/L2-Norm)
1/0.0000 

In [5]:
rky.icrfs.GetIdx()

0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
11    11
12    12
13    13
14    14
15    15
16    16
17    17
18    18
19    20
20    21
21    22
22    23
23    24
24    25
25    26
26    27
27    30
28    31
29    32
30    33
31    34
32    35
33    36
34    40
35    41
36    42
37    43
38    44
39    45
40    50
41    51
42    52
43    53
44    54
45    60
46    61
47    62
48    63
49    70
50    71
51    72
52    80
53    81
54    90
dtype: int64

In [6]:
rky.icrfs.GetCVStatistics()

X:    intercept  accident_period_2002  accident_period_2003  \
0          1                     0                     0   
1          1                     1                     0   
2          1                     0                     1   
3          1                     0                     0   
4          1                     0                     0   

   accident_period_2004  accident_period_2005  accident_period_2006  \
0                     0                     0                     0   
1                     0                     0                     0   
2                     0                     0                     0   
3                     1                     0                     0   
4                     0                     1                     0   

   accident_period_2007  accident_period_2008  accident_period_2009  \
0                     0                     0                     0   
1                     0                     0                     0

,n_run,train_mse,test_mse,train_mae,test_mae,train_mape,test_mape,train_msle,test_msle
0,1,0.602362,0.899664,0.656229,0.735559,1.360161,0.906413,NaN,NaN
1,2,0.504740,1.184415,0.591539,0.844762,1.236848,1.137061,NaN,NaN
2,3,0.311430,1.463392,0.504096,0.955620,1.746443,1.828788,NaN,NaN
3,4,0.314188,1.337858,0.521403,0.880916,2.211219,1.949156,NaN,NaN
4,5,0.284673,1.095323,0.492531,0.813183,0.888326,1.466822,NaN,NaN


^^ this doesn't work when using a real triangle?

In [7]:
rky.icrfs.LassoFeatureImportance()

,0.0000,0.0025,0.0050,0.0075,0.0100,0.0125,0.0150,0.0175,0.0200,0.0225,...,0.2600,0.2625,0.2650,0.2675,0.2700,0.2725,0.2750,0.2775,0.2800,0.2825
0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,105.0,106.0,107.0,108.0,109.0,110.0,111.0,112.0,113.0,114.0
1,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,105.0,106.0,107.0,108.0,109.0,110.0,111.0,112.0,113.0,114.0
2,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,105.0,106.0,107.0,108.0,109.0,110.0,111.0,112.0,113.0,114.0
3,114.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,104.0,105.0,106.0,107.0,108.0,109.0,110.0,111.0,112.0,113.0
4,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,105.0,106.0,107.0,108.0,109.0,110.0,111.0,112.0,113.0,114.0
5,2.0,1.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,105.0,106.0,107.0,108.0,109.0,110.0,111.0,112.0,113.0,114.0
6,2.0,1.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,105.0,106.0,107.0,108.0,109.0,110.0,111.0,112.0,113.0,114.0
7,2.0,1.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,105.0,106.0,107.0,108.0,109.0,110.0,111.0,112.0,113.0,114.0
8,114.0,113.0,112.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,...,102.0,103.0,104.0,105.0,106.0,107.0,108.0,109.0,110.0,111.0
9,114.0,113.0,112.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,...,102.0,103.0,104.0,105.0,106.0,107.0,108.0,109.0,110.0,111.0


In [8]:
rky.icrfs.tri.ult()

Accident Period
2001-01-01    3901463.0
2002-01-01    5433719.0
2003-01-01    5378826.0
2004-01-01    5297906.0
2005-01-01    4858200.0
2006-01-01    5111171.0
2007-01-01    5660771.0
2008-01-01    6784799.0
2009-01-01    5642266.0
2010-01-01    4969825.0
Name: Chain Ladder Ultimate Loss, dtype: float64

In [9]:
rky.icrfs.Ultimate()

accident_period
2001-01-01    2567142.0
2002-01-01    3706315.0
2003-01-01    3638545.0
2004-01-01    3629865.0
2005-01-01    3506981.0
2006-01-01    3807868.0
2007-01-01    4326249.0
2008-01-01    4866519.0
2009-01-01    4457979.0
2010-01-01    4296988.0
Name: Log-linear Ultimate, dtype: float64

In [10]:
rky.icrfs.TrendPlot()

In [15]:
#test 
rky.icrfs.FitPlot('acc')

In [12]:
rky.icrfs.ResidualPlot(color_col='acc')

$\text{AIC} = 2k - 2l$

In [13]:
# k - number of parameters:
rky.icrfs.GetP()

37

In [14]:
# l - loglikelihood:
# log(likelihood)


In general

Objective function of elastic net regularization:

$$
\min_{\beta} \frac{1}{2n} \sum_{i=1}^n \left( y_i - \beta_0 - \mathbf{x}_i^T\beta \right)^2 + \alpha \lambda \|{\beta}\|^1 + \frac{\alpha(1-\lambda)}{2} \|\beta\|_2^2
$$

Here $\alpha$ is the strength of the regularization, $\lambda$ is the portion of the regularization that is $\ell_1$-norm, and $k$ is the number of parameters in the model.

